In [2]:
import uproot
import h5py
import numpy as np
import os
import fnmatch
import awkward as awk
import glob

In [3]:
import sys
sys.path.append('../src/')

In [5]:
from heplot import plotting as heplt
import selection as sele
import string_constants as stco
import reader as read

In [7]:
# read electron files
path_ee = glob.glob(os.path.join(stco.generator_in_dir_lightjet,'*'+stco.ds_ids['ee'][0]+'*'))[0]

In [8]:
path_ee

'/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2/mc20_Run2/Ntuples/PeriodE/ZJets_MG/user.ltoffoli.mc20_13TeV.506193.MGPy8EG_3jets_HT2bias_BFilter.deriv.DAOD_PHYS.e8382_s3681_r13145_p5631.toffoGN2_v1_output.root'

In [9]:
fnames_ee = os.listdir(path_ee)

In [10]:
fnames_ee

['user.ltoffoli.35090069._000001.output.root',
 'user.ltoffoli.35090069._000002.output.root',
 'user.ltoffoli.35094310._000001.output.root',
 'user.ltoffoli.35094310._000002.output.root']

In [11]:
feature_names = ['el_e','mu_e','el_charge','mu_charge','el_pt','mu_pt', 'el_phi','mu_phi',\
                      'el_eta', 'mu_eta', 'jet_e', 'jet_GN2_pu', 'jet_GN2_pb', 'jet_GN2_pc']

In [38]:
batch_sz = 1000

In [45]:
def sample_generator(path, batch_sz):
    
    samples_concat = None

    # for ff in uproot.iterate([pp+'/*.root:nominal' for pp in path_ee]):
    for next_chunk in uproot.iterate([path+'/*.root:nominal'],feature_names):

        #print(len(next_chunk))

        # select lightjets
        selected = sele.select_lightjets(next_chunk)

        if samples_concat is None: 
            samples_concat = selected
        else:
            samples_concat = awk.concatenate([samples_concat,selected])

        while(len(samples_concat) >= batch_sz):
            samples_batch = samples_concat[:batch_sz]
            samples_concat = samples_concat[batch_sz:]
            yield samples_batch

        # if data left, yield it
        if len(samples_concat) > 0:
            yield samples_concat

In [46]:
gen = sample_generator(path_ee, batch_sz)

In [48]:
i = 0
for sample in gen:
    print(len(sample))
    print(sample)
    i = i+1
    if i > 10: break

1000
[{el_e: [1.51e+05, 8.89e+04], mu_e: [], ... jet_GN2_pc: [0.111, 0.000377, 0.132]}]
1000
[{el_e: [2.8e+05, 2.69e+05], mu_e: [], ... jet_GN2_pc: [0.207, 0.181, 0.0364]}]
1000
[{el_e: [1.44e+05, 1.42e+05], mu_e: [], ... jet_GN2_pc: [0.176, 0.0219, 0.224]}]
1000
[{el_e: [2.66e+05, 4.16e+05], mu_e: [], ... 1], jet_GN2_pc: [5.36e-06]}]
1000
[{el_e: [6.94e+04, 4.52e+04], mu_e: [], ... 0.157, 0.127, 0.136, 0.0345]}]
1000
[{el_e: [1.43e+05, 3.91e+04], mu_e: [], ... 0.000352, 0.00204, 0.244, 0.177, 0.217]}]
1000
[{el_e: [1.19e+05, 4.8e+04], mu_e: [], ... 0.201, 0.09, 1.71e-05, 0.189, 0.0291]}]
1000
[{el_e: [1.74e+05, 4.23e+04], mu_e: [], ... jet_GN2_pc: [0.388, 0.00354, 0.185]}]
1000
[{el_e: [7.35e+05, 1.99e+05], mu_e: [], ... jet_GN2_pc: [0.109, 0.0906, 0.159]}]
1000
[{el_e: [8.98e+04, 3.9e+04], mu_e: [], ... jet_GN2_pc: [0.162, 0.16, 0.267]}]
1000
[{el_e: [1.72e+05, 4.64e+04], mu_e: [], ... 1], jet_GN2_pc: [2.47e-05]}]


In [ ]:
i = 0
for next_chunk in uproot.iterate([path+'/*.root:nominal']):
    